# Partition

It is very important to understand the partition concept. There are 4 major components of a partition concept in Fugue. We use an example dataframe to explain the concept

item|a:int|b:int|
---|---|---|
0  |0  |0  |
1  |1  |1  |
2  |2  |2  |
3  |0  |2  |
4  |2  |1  |


## Number Of Partitions

It specifies **at most** how many buckets for these items. For example if `partition_num=2` then it's possible item 0,2 are in bucket 0 and item 1,3,4 are in bucket 1.

If in a partition you only specify number of partitions, it indicates you want to `reshuffle` the data to number of partitions so in the next step you can process them with controlled concurrency. Notice that Spark has more granular concepts, `repartition` and `coalesce`, you can read [this great article](https://blog.knoldus.com/apache-spark-repartitioning-v-s-coalesce/) for details.

In current Fugue SparkExecutionEngine, we only use `repartition` because we notice that `reshuffle` is no longer bad in Spark. It performs extremely well in most cases, and data with `reshuffle` is a lot more balanced than `coalesce` without noticeable performance issue. The advantage of `repartition` has been proven with numerous production use cases. Plus `coalesce` seems to be more inclined to `push-down`, causing a lot of hard-to-find performance issues. So to avoid confusion and inconsistency, we only use `repartition` in SparkExecutionEngine. That being said, you can inherit from built-in SparkExecutionEngine and modify to use `coalesce` if you feel that is useful to you.

**All Fugue ExecutionEngine follows this rule on number of partitions**:
* **<=0** means I don't want to repartition to current dataframe with an explicit number. The ExecutionEngine can decide whether to do nothing or reshuffle to new buckets.
* **==1** means I want all data to move to a single partition on a single machine (physical location)
* **>1** means I want to reshuffle the data to the number of buckets

The underlying computing frameworks may have inconsistent behavior on this (you have to be careful if moving away from Fugue), but on Fugue ExecutionEngine level, they are consistent.


## Partition Keys

It specifies the keys to partiton on. You can think the number of partitions is to define the number of **physical** partitions, and partition keys is to define the **logical** partitions. All items with a same set of keys will be moved to a single physical location to process and partition keys is to tell the framework within this physical partition, I will separate the data by certain keys and process them separately.

For example, if I only specify `partition by a`, then the original dataframe can be partitioned to two physical parts

item|a:int|b:int|partition
---|---|---|---
0  |0  |0  |0
3  |0  |2  |0
1  |1  |1  |1
2  |2  |2  |1
4  |2  |1  |1

And in partition 1, if sorting by key `a`, we will be able to find the boundary of each logical partition, and process them separately. If you also specify number of partitions in the mean time, that means you want to control the concurrency as well, this may or may not take effect and if it takes effect, it may or may not be positive. This is something you need to tune a bit.


## Presort

In each **physical** partition, in order to figure out the **logical** partitions, you may have to sort first. And after you get the **logical** partitions, you may also need to sort inside that partition on some other key before process. This additional sort can take a lot of time. By specifying `presort` you actually combine this additional sort with the necessary sort to figure out the logical partition. So with a single sort, you can partition the data and also make each partition sorted as expected. `presort` must not overlap with `partition keys`, and you can also specify whether to sort ascendingly or descendingly. Taking the previsoue example, if `partition by a presort b asc`, it becomes:

item|a:int|b:int|partition
---|---|---|---
0  |0  |0  |0
3  |0  |2  |0
1  |1  |1  |1
4  |2  |1  |1
2  |2  |2  |1


## Reshuffle Rules (Hint)

Notice this is a hint, it does not require every ExecutionEngine to strictly follow. Currently only SparkExecutionEngine supports it. It has no effect on other engines.

Reshuffle has to follow certain rules, no rule is good for everything, especially for small dataframes where for each item it takes a lot of time to compute, you need to be very careful about it.

Currently 3 rules are supported: `HASH` (default), `RAND`, `EVEN`.

There are 2 cases:

### With No Partition Keys

You only specify number of partitions with a rule hint. `HASH` will repartition the items by hash code, it's deterministic and fast, but it can't guarantee perfect eveness. For example, if I want to hash repartition the dataframe to `5` partitions, I may get this:

item|a:int|b:int|partition
---|---|---|---
0  |0  |0  |0
3  |0  |2  |1
1  |1  |1  |1
2  |2  |2  |3
4  |2  |1  |4

Partition 1 will have 2 values but partition 2 will be empty. If each item takes 1 hour to process, you have to spend at least 2 hours to process all because of the uneveness. `RAND` is random repartition, it's not deterministic, and can't guarantee eveness either. But `EVEN` repartition can. With this rule, it's guaranteed 1 and only 1 item in each physical partition. But there is cost to do so, it has to cache the dataframe and index it and redistribute. For this case if each item takes 1 hour to process, and with only 5 items, the cost of `EVEN` repartition is ignorable, but if the dataframe is very big, it's uncertain if it's worth it. On the other hand, if the dataframe is much larger than number of partitions, both other two rules will generate relatively even partitions. Here is a table of comparison

| Feature | `HASH` | `RAND` | `EVEN` |
| --- | --- | --- | --- |
| Speed | fast (map, shuffle) | fast (map, shuffle) | slow (map, reduce, map, shuffle) |
| Space | low | low | high (need cache the dataframe first) |
| Deterministic | yes | no | yes |
| Eveness (small data) | bad | random | good (strict eveness) |
| Eveness (big data) | good | good | best (but worth it?) |

### With Partition Keys

With partition keys, `RAND` will become `HASH`. `EVEN` will guarantee eveness on the **logical** level. For the same example, if `partition num=3 by a`, then `HASH` may have this:

item|a:int|b:int|partition
---|---|---|---
0  |0  |0  |0
3  |0  |2  |0
1  |1  |1  |1
2  |2  |2  |1
4  |2  |1  |1

But `EVEN` will make each **logical** partition onto its own **physical** partition. Because there are 3 keys, and we specify the number of partitions to be 3. 

item|a:int|b:int|partition
---|---|---|---
0  |0  |0  |0
3  |0  |2  |0
1  |1  |1  |1
2  |2  |2  |2
4  |2  |1  |2

If we set the number to be larger than 3, it's still even. If there are 100 logical partitions and partition num=50, then each physical partition will strictly contain 2 logical partitions.

`EVEN` is the best for small size but expensive computations.



# PartitionSpec in Fugue

Here are examples to initialize PartitionSpec in Fugue. It's a widely used data structure

In [8]:
from fugue.collections.partition import PartitionSpec

assert PartitionSpec().empty # empty partition spec means no operation needed, it can be the default value
PartitionSpec(num=4)
PartitionSpec(algo="even",num=4,by=["a","b"],presort="c,d desc") # c,d desc == c asc, d desc

# you can use expression in num, ROWCOUNT can be used to indicate using the row count of the dataframe to operate on
# if a df has 1000 rows, this means I want to even partition it to 10 rows per partition
PartitionSpec(algo="even",num="ROWCOUNT/10") 